In [1]:
from __future__ import print_function
"""
Tutorial: A reference implementation of non-hermitian cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# Non-Hermitian Cavity Quantum Electrodynamics Configuration Interaction Singles (NH-CQED-CIS) 

## I. Theoretical Overview

This tutorial builds from the Psi4Numpy tutorial on canonical CIS for molecular systems, that implements the configuration interaction singles method in a spin-adapted basis. The groundwork for working in the spin orbital notation has been laid out in "Introduction to the Spin Orbital Formulation of Post-HF methods" [tutorial](../CQED-Methods/cis_spatial_orbitals.ipynb). It is highly recommended to work through that introduction before starting this tutorial. 

We propose a novel formulation of ab initio CQED that couples an ab initio molecular Hamiltonian to a non-Hermitian photonic Hamiltonian to capture the finite lifetime and energy uncertainty of nanoconfined optical resonances.   As a first realization of this theory, will develop a configuration interaction singles approach for the energy eigenstates of this non-Hermitian polaritonic Hamiltonian, which will allow us to leverage tools of modern quantum chemistry, including analytic evaluation of forces and nonadiabatic couplings, to study polaritonic structure and reactivity.  This will pave the way for future developments of a hierarchy of CI-based approaches that can be systematically improved in terms of their accuracy, and are also interoperable with the powerful machinery of quantum chemistry codes.  In particular, we envision that Complete Active Space CI theory, which has been a particularly promising approach for simulating photochemistry, can be merged with NH-CQED to provide a powerful tool for simulating polaritonic chemistry.
In the following presentation of NH-CQED-CIS theory, we will consider only a single photonic degree of freedom for notational simplicity, but generalizations to additional photonic modes is feasible.  The total polaritonic Hamiltonian operator may be written as
\begin{equation}
    \hat{H} = \sum_{pq} h_{pq} \hat{a}_p^{\dagger} \hat{a}_q + 
    \sum_{pqrs} V_{pq}^{rs} \hat{a}_p^{\dagger} \hat{a}_q^{\dagger} \hat{a}_s \hat{a}_r
    + \hbar \left( \omega - i \frac{\gamma}{2} \right)\hat{b}^{\dagger} \hat{b} 
    + \sum_{pq} \hbar g_{pq} \hat{a}_p^{\dagger} \hat{a}_q \left(\hat{b}^{\dagger} + \hat{b} \right),
\end{equation}
where $h_{pq}$ includes the molecular electronic kinetic energy and electron-nuclear potential
integrals, $V_{pq}^{rs}$ denotes the molecular 2-electron repulsion integrals, 
$\hat{a}_p^{\dagger}$ and $\hat{a}_q$ denotes the electronic creation and annihilation 
operators, $\omega$ and $\gamma$ denote the energy and lifetime of the photonic degree of 
freedom, $\hat{b}^{\dagger}$ and $\hat{b}$ denote the photonic raising and lowering operators, and $g_{pq}$ denotes the coupling between the photonic and electronic degrees of freedom. 


In [1]:
# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
import scipy.linalg as la

# ==> Set Basic Psi4 Options <==

# Memory specifications
psi4.set_memory(int(2e9))
numpy_memory = 2

# Output options
psi4.core.set_output_file('output.dat', False)

Define the molecule!

In [2]:
mol = psi4.geometry("""
0 1
O
H 1 1.1
H 1 1.1 2 104
symmetry c1
""")

psi4.set_options({'basis':        'sto-3g',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'save_jk': True,
                  'e_convergence': 1e-8,
                  'd_convergence': 1e-8})

We use Psi4 to compute the RHF energy and wavefunction and store them in variables `scf_e` and `scf_wfn`. We also check the memory requirements for computation:

In [3]:
# Get the SCF wavefunction & energies
scf_e, wfn = psi4.energy('scf', return_wfn=True)

# ==> Nuclear Repulsion Energy <==
E_nuc = mol.nuclear_repulsion_energy()

# Check memory requirements
nmo = wfn.nmo()
I_size = (nmo**4) * 8e-9
print('\nSize of the ERI tensor will be %4.2f GB.\n' % I_size)
memory_footprint = I_size * 1.5
if I_size > numpy_memory:
    psi4.core.clean()
    raise Exception("Estimated memory utilization (%4.2f GB) exceeds allotted \
                     memory limit of %4.2f GB." % (memory_footprint, numpy_memory))


Size of the ERI tensor will be 0.00 GB.



We first obtain orbital information from our wavefunction. We also create an instance of MintsHelper to help build our molecular integrals:

In [6]:
# Create instance of MintsHelper class
mints = psi4.core.MintsHelper(wfn.basisset())

# Grab data from wavfunction

# number of doubly occupied orbitals
ndocc   = wfn.nalpha()

# total number of orbitals
nmo     = wfn.nmo()

# number of virtual orbitals
nvirt   = nmo - ndocc

# orbital energies
eps     = np.asarray(wfn.epsilon_a())

# occupied orbitals:
Co = wfn.Ca_subset("AO", "OCC")

# virtual orbitals:
Cv = wfn.Ca_subset("AO", "VIR")

# grab all transformation vectors and store to a numpy array!
C = np.asarray(wfn.Ca())
print(C)

[[ 9.94434590e-01  2.39158855e-01  3.10144378e-15  9.36832397e-02
  -1.00207956e-17  1.11639917e-01 -2.08134820e-15]
 [ 2.40970413e-02 -8.85735625e-01 -1.58812804e-14 -4.79585843e-01
  -4.20179491e-17 -6.69578990e-01  1.26244160e-14]
 [ 3.16154851e-03 -8.58961837e-02  2.60123279e-14  7.47431392e-01
   2.64341320e-16 -7.38488591e-01  1.47262736e-14]
 [ 1.16161167e-16  1.09714926e-16  1.14693814e-15 -2.60451346e-16
   1.00000000e+00  1.08793317e-16 -4.64654018e-17]
 [ 8.50327039e-17 -2.11900024e-15  6.07284839e-01 -2.00514833e-14
  -5.91543715e-16  1.85055877e-14  9.19234239e-01]
 [-4.59374285e-03 -1.44039545e-01 -4.52997745e-01  3.29471186e-01
   7.67062335e-16  7.09849461e-01  7.32460618e-01]
 [-4.59374285e-03 -1.44039545e-01  4.52997745e-01  3.29471186e-01
  -5.32877393e-16  7.09849461e-01 -7.32460618e-01]]


Let's first take care of the molecule-photon coupling terms.  A 
dipole-dipole coupling potential provides a reasonable approximation to this term:
\begin{equation}
g_{pq} = \frac{1}{r^3} \mu_{pq} \cdot \mu_s - \frac{3}{r^5}\left(\mu_{pq} \cdot r \right) \left(\mu_s \cdot r \right),
\end{equation}
where $\mu_{pq}$ are the the dipole integrals in the MO basis.

We can transform the dipole integrals from the AO to the MO basis in the following way:
\begin{equation}
{\bf \mu}^{\xi} = {\bf C}^T {\bf \mu}^{\xi}_{ao} {\bf C},
\end{equation}
where ${\bf \mu}^{\xi}_{ao}$ represents the $\xi$ component of the dipole integrals in the AO basis and ${\bf C}$
represents the matrix of transformation vectors that go from AOs to MOs.  Note here 
$\xi$ can be $x$, $y$, or $z$.  At first pass, we will consider only the $z$ component, which means that 
the molecule will only couple to photons polarized along the $z$ axis.

In [10]:
# ==> array of dipole integrals <==
# start by assuming photon polarized along z so only
# need z-component of molecular dipole integrals!
mu_ao_z = np.asarray(mints.ao_dipole()[2])

# transform to the MO basis
mu_z = np.dot(C.T, mu_ao_z).dot(C)

### if you want to see the arrays, uncomment here!
print("z-component of the AO dipole matrix")
print(mu_ao_z)
print("z-component of the MO dipole matrix")
print(mu_z)
print(len(mu_z))



z-component of the AO dipole matrix
[[ 0.14322583  0.03390212 -0.05079193  0.          0.          0.00375868
   0.00375868]
 [ 0.03390212  0.14322583 -0.64117284  0.          0.         -0.14997188
  -0.14997188]
 [-0.05079193 -0.64117284  0.14322583  0.          0.         -0.33409068
  -0.33409068]
 [ 0.          0.          0.          0.14322583  0.          0.
   0.        ]
 [ 0.          0.          0.          0.          0.14322583  0.10895216
  -0.10895216]
 [ 0.00375868 -0.14997188 -0.33409068  0.          0.10895216 -1.1365489
  -0.20657896]
 [ 0.00375868 -0.14997188 -0.33409068  0.         -0.10895216 -0.20657896
  -1.1365489 ]]
z-component of the MO dipole matrix
[[ 1.42888805e-01  5.72298702e-03 -1.71388639e-15 -4.73272362e-02
  -1.60870876e-18  4.50652387e-02 -8.10778196e-16]
 [ 5.72298702e-03 -1.37544374e-01  2.37488130e-14  7.23171607e-01
   2.68944606e-16  4.73964125e-02 -1.06362362e-15]
 [-1.71385300e-15  2.37624545e-14 -4.48742027e-01  1.76419171e-14
   8.04852423

Restricting the polarization of the photon to the z-direction means we need 
only worry about the scalar separation (along the z-axis) between the center-of-mass of
the molecular transition dipoles and the photon transition dipole ($r_z$) and the 
$z$-component of the photon transition dipole $\mu_{s,z}$.
\begin{equation}
g_{pq} = \frac{1}{r_z^3} \mu_{z,pq} \cdot \mu_{z,s} - \frac{3}{r_z^5}\left(\mu_{z,pq} \cdot r_z \right) \left(\mu_{s,z} \cdot r_z \right),
\end{equation}

In [29]:
def compute_coupling(mu_mol, mu_phot, r):
    ''' write code to loop through all 
        elements of mu_mol and compute the coupling
        to the (single) mu_phot '''
    # dimension of mu_mol
    dim = len(mu_mol)
    # allocate g
    g = np.zeros((dim,dim))
    # take care of constant eleents first!
    # 1 / r^3
    oer3  = 1 / r ** 3
    # 3 / r^5
    toer5 = 3 / r ** 5
    # mu_s * r
    mu_s_dot_r = mu_phot * r
    
    # now loop through all the elements of mu and compute the first and second terms
    for p in range(0, dim):
        for q in range(0, dim):
            term_1 = oer3 * mu_mol[p,q] * mu_phot
            term_2 = toer5 * mu_mol[p,q] * r * mu_s_dot_r
            g[p,q] = term_1 - term_2
            
    return g
            
### now actually compute g matrix
# define r in atomic units... 1 atomic unit = 0.529 Angstomrs, so r = 20 a.u. is ~1 nm
r = 10
# define mu_phot in atomic units... plasmon resonance in a ~5 nm Ag particle on the order mu_phot = 100 a.u.
mu_s = 100

gpq = compute_coupling(mu_z, mu_s, r)

print("printing g matrix")
print(gpq)

    
    

printing g matrix
[[-2.85777610e-02 -1.14459740e-03  3.42777277e-16  9.46544724e-03
   3.21741752e-19 -9.01304775e-03  1.62155639e-16]
 [-1.14459740e-03  2.75088747e-02 -4.74976259e-15 -1.44634321e-01
  -5.37889211e-17 -9.47928250e-03  2.12724724e-16]
 [ 3.42770600e-16 -4.75249091e-15  8.97484054e-02 -3.52838342e-15
  -1.60970485e-16  1.15359728e-15 -1.40655270e-01]
 [ 9.46544724e-03 -1.44634321e-01 -3.52865317e-15 -7.65735964e-03
   2.09862576e-17  1.18711747e-01  2.24475376e-15]
 [ 3.21741752e-19 -5.37889211e-17 -1.60970485e-16  2.09862576e-17
  -2.86451654e-02  3.50187131e-17  1.86846722e-16]
 [-9.01304775e-03 -9.47928250e-03  1.15106695e-15  1.18711747e-01
   3.50187131e-17  1.70676235e-01 -1.29234091e-15]
 [ 1.62196645e-16  2.13717932e-16 -1.40655270e-01  2.24265051e-15
   1.86846722e-16 -1.29896094e-15  1.16679848e-01]]


## NH-CQED-CIS

In CIS, only single excitations from the occupied (indices i,j,k...) to the virtual (indices a,b,c...) orbitals are included. As a result, CIS gives transition energies to an excited state. 

The polaritonic energy eigenfunctions for state $I$ in the
CQED-CIS ansatz can be written as 
\begin{equation}
\Psi_I = c_{0,0} |\Phi_0\rangle |0\rangle + 
c_{0,1} |\Phi_0\rangle |1\rangle +
c^a_{i,0} |\Phi_i^a\rangle |0\rangle +
c^a_{i,1} |\Phi_i^a\rangle |1\rangle 
\end{equation}

Assuming we are using the spin-adapted determinants based on canonical Hartree-Fock orbitals ($\{\mathrm{\psi_p}\}$) with orbital energies $\{\epsilon_p\}$, we can build a shifted CIS Hamiltonian matrix:

\begin{equation}
\tilde{\textbf{H}} = \textbf{H} - E_0 \textbf{I} = 
[\langle s| \langle \Phi_P | \hat{H} - E_0|\Phi_Q \rangle | t \rangle ]
\end{equation}
where
\begin{equation}
\Phi_P, \Phi_Q \in {\Phi_i^a} \; {\rm and} \; |s\rangle, |t\rangle \in {|0\rangle, |1\rangle}
\end{equation}

and where $E_0$ is the ground state Hartree-Fock state energy given by $\langle \Phi | \hat{H}_e|\Phi \rangle$,
where $\hat{H}_e$ contains only the molecular electronic terms.

The matrix elements of this shifted CIS Hamiltonian matrix contains three contributions:

\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H} - E_0|\Phi_j^b \rangle | t \rangle = 
\langle s | \langle \Phi_i^a | \hat{H}_e - E_0|\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_i^a | \hat{H}_p |\Phi_j^b \rangle | t \rangle
+ \langle s | \langle \Phi_i^a | \hat{H}_{ep}|\Phi_j^b \rangle | t \rangle.
\end{equation}
The first term is similar to the ordinary CIS matrix elements with the requirement that the photonic
bra and ket states be identical:
\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H_e} - E_0|\Phi_j^b \rangle | t \rangle = \left((\epsilon_a - \epsilon_i)\delta_{ij} \delta_{ab}
+ 2(ia|jb) - (ij|ab) \right) \delta_{st}.
\end{equation}
The second term vanishes unless both the photonic and molecular bra and ket states are identical:
\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H}_p |\Phi_j^b \rangle | t \rangle = \delta_{st} \delta_{ij} \delta_{ab} \left( \hbar \omega + i\frac{\gamma}{2}\right)\sqrt{s}.
\end{equation}
The third term couples particular transitions between photonic and molecular bra and ket states:
\begin{equation}
\langle s | \langle \Phi_i^a | \hat{H}_{ep}|\Phi_j^b \rangle | t \rangle =
\left( 1 - \delta_{st} \right) \left( g_{ab} \delta_{ij} - g_{ij} \delta_{ab} \right)
\end{equation}

This then becomes a standard eigenvalue equation from which we can solve for the excitation energies and the wavefunction expansion coefficients:

\begin{equation}
\tilde{\textbf{H}} \textbf{c}_K = \Delta E_K\textbf{c}_K, \,\Delta E_K = E_K - E_0
\end{equation}



## II. Implementation

We will prepare the ordinary CIS quantities as we did in [this tutorial](../CQED-Methods/cis_spatial_orbitals.ipynb):

We now transform the 2-electron integral from the AO basis into the MO basis using the coefficients:


In [30]:
# build the (ov|ov) integrals:
ovov = np.asarray(mints.mo_eri(Co, Cv, Co, Cv))

# build the (oo|vv) integrals:
oovv = np.asarray(mints.mo_eri(Co, Co, Cv, Cv))

# strip out occupied orbital energies, eps_o spans 0..ndocc-1
eps_o = eps[:ndocc]

# strip out virtual orbital energies, eps_v spans 0..nvirt-1
eps_v = eps[ndocc:]
### if you want to print these arrays, go ahead and uncomment!
#print(oovv)
#print(ovov)
#print(eps_o)
#print(eps_v)

In [31]:
# Initialize CIS matrix.
# The dimensions are the number of possible single excitations
HCIS = np.zeros((ndocc * nvirt * 2, ndocc * nvirt * 2))
print(HCIS)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [32]:
# photon frequency
om = 0.58

for i in range(0, ndocc):
    for a in range(0, nvirt):
        for s in range(0,2):
            ias = 2*(i*nvirt + a) + s
            print(ias)
            #print("i,a,s,ias:",i,a,s,ias)
            
            for j in range(0, ndocc):
                for b in range(0, nvirt):
                    for t in range(0,2):
                        jbt = 2*(j*nvirt + b) + t
                        #print(jbt)
                        #print("j,b,t,jbt:",j,b,t,jbt)
                        # ordinary CIS term, contributes whenever s == t
                        term1 = (2 * ovov[i, a, j, b] - oovv[i, j, a, b]) * (s == t)
                        # ordinary CIS term for when i==j and a==b
                        term2 = (eps_v[a] - eps_o[i]) * (s == t) * (i == j) * (a == b)
                        # ordinary photonic term for when s == t and i == j and a == b
                        term3 = np.sqrt(s) * om * (i == j) * (a == b) * (s == t)
                        # coupling term between molecular and photonic states
                        term4 = (1 - (s == t)) * (gpq[a, b] * (i == j) - gpq[i,j] * (a == b))
                        HCIS[ias, jbt] = term1 + term2 + term3 + term4
                    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [33]:
#print(HCIS)
ECIS, CCIS = np.linalg.eig(HCIS)

print("No coupling")
print(ECIS_NC)

print("Yes coupling")
print(ECIS)

No coupling
[20.01097938 20.01097938 20.05053189 20.05053189  0.91012162  1.30078511
  0.91012162  0.3564617   0.50562823  0.41607167  1.30078511  0.50562823
  1.32576198  1.32576198  0.65531837  0.65531837  0.55519181  0.55519181
  0.3564617   0.41607167]
Yes coupling
[20.01098195 20.0451573  20.63591681 20.5909862   1.90821182  1.92165263
  1.50406019  1.3403502   1.41371434  0.49204137  0.63205925  0.53763171
  1.23720223  0.89519646  1.03319796  1.07027635  0.35645946  0.41068277
  1.00146052  0.93646399]


## References
1. Background paper:
 >"Toward a systematic molecular orbital theory for excited states"
[[Foresman:1992:96](http://pubs.acs.org/doi/abs/10.1021/j100180a030)] J. B. Foresman, M. Head-Gordon, J. A. Pople, M. J. Frisch, *J. Phys. Chem.* **96**, 135 (1992).


2. Algorithms from: 
	> [[CCQC:CIS](https://github.com/CCQC/summer-program/tree/master/7)] CCQC Summer Program, "CIS" accessed with https://github.com/CCQC/summer-program/tree/master/7.
    